In [ ]:
import pandas as pd  # Importing the pandas library for data manipulation and analysis.
import numpy as np  # Importing the numpy library for numerical operations.
import boto3  # Importing the boto3 library to interact with AWS services.
import sagemaker  # Importing the sagemaker library for building, training, and deploying machine learning models on AWS SageMaker.
import time  # Importing the time library for time-related tasks.
import json  # Importing the json library for JSON parsing.
import io  # Importing the io library for handling I/O stream operations.
from io import StringIO  # Importing StringIO from io for in-memory text streams.
import base64  # Importing the base64 library for encoding and decoding Base64 data.
import pprint  # Importing the pprint library for pretty-printing Python data structures.
import re  # Importing the re library for regular expression operations.

from sagemaker.image_uris import retrieve  # Importing the retrieve function from sagemaker.image_uris for retrieving container image URIs.

sess = sagemaker.Session()  # Creating a SageMaker session object for managing interactions with the SageMaker service.
write_bucket = sess.default_bucket()  # Retrieving the default S3 bucket for the SageMaker session.
write_prefix = "fraud-detect-demo"  # Defining the S3 prefix for writing data related to the fraud detection demo.

region = sess.boto_region_name  # Retrieving the AWS region name associated with the SageMaker session.
s3_client = boto3.client("s3", region_name=region)  # Creating an S3 client for interacting with Amazon S3.
sm_client = boto3.client("sagemaker", region_name=region)  # Creating a SageMaker client for interacting with the SageMaker service.
sm_runtime_client = boto3.client("sagemaker-runtime")  # Creating a SageMaker Runtime client for invoking deployed models.
sm_autoscaling_client = boto3.client("application-autoscaling")  # Creating an Application Auto Scaling client for managing auto-scaling policies.

sagemaker_role = sagemaker.get_execution_role()  # Retrieving the IAM role for SageMaker execution.

# S3 locations used for parameterizing the notebook run
read_bucket = "sagemaker-sample-files"  # Defining the S3 bucket for reading sample files.
read_prefix = "datasets/tabular/synthetic_automobile_claims"  # Defining the S3 prefix for reading synthetic automobile claims data.
model_prefix = "models/xgb-fraud"  # Defining the S3 prefix for the XGBoost fraud detection model.

data_capture_key = f"{write_prefix}/data-capture"  # Defining the S3 key for data capture during model inference.

# S3 location of trained model artifact
model_uri = f"s3://{read_bucket}/{model_prefix}/fraud-det-xgb-model.tar.gz"  # Defining the S3 URI of the trained XGBoost model artifact.

# S3 path where data captured at endpoint will be stored
data_capture_uri = f"s3://{write_bucket}/{data_capture_key}"  # Defining the S3 URI for storing data captured at the inference endpoint.

# S3 location of test data
test_data_uri = f"s3://{read_bucket}/{read_prefix}/test.csv"  # Defining the S3 URI for the test data.